In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import os
from collections import defaultdict
import geopandas as gpd
import geojson
from geojson import MultiPoint
import json
from sklearn.cluster import Birch

## Read the data and process it

In [2]:
df = pd.read_csv("Data/NYPD_Complaint_Data_2013_plus.csv", low_memory=False)

In [3]:
def get_year(x):
    return x.year

def get_month(x):
    return x.month

def get_day(x):
    return x.day

def get_hour(x):
    return x.hour

def get_minute(x):
    return x.minute

def get_week(x):
    return x.isocalendar()[1]

def get_weekday(x):
    return x.isoweekday()#int(x.strftime("%w"))

def convert_to_time(x):
    if type(x) == str:
        return dt.datetime.strptime(x, "%H:%M:%f")
    else:
        return None

def merge_date_and_time(d, t):
    if type(d) == str and type(t) == str:
        return dt.datetime.strptime("{0} {1}".format(d, t), "%m/%d/%Y %H:%M:%f")
    else:
        print(d)
        print(t)
        return None

# Drop all records with no recorded date or time
df = df[pd.notna(df["CMPLNT_FR_DT"])]
df = df[pd.notna(df["CMPLNT_FR_TM"])]
df["Date"] = pd.Series(np.vectorize(dt.datetime.strptime)(df["CMPLNT_FR_DT"].values, "%m/%d/%Y"))
df = df[pd.notna(df["Date"])]
df["Time"] = pd.Series(np.vectorize(convert_to_time)(df["CMPLNT_FR_TM"].values))

df["Datetime"] = pd.Series(np.vectorize(merge_date_and_time)(df["CMPLNT_FR_DT"].values, df["CMPLNT_FR_TM"].values))
df = df[pd.notna(df["Datetime"])]
df = df[df["Datetime"] >= dt.datetime(2006,1,1)]
df["Year"] = pd.Series(np.vectorize(get_year)(df["Date"].values))
df["Month"] = pd.Series(np.vectorize(get_month)(df["Date"].values))
df["Day"] = pd.Series(np.vectorize(get_day)(df["Date"].values))
df["Hour"] = pd.Series(np.vectorize(get_hour)(df["Datetime"].values))
df["Minute"] = pd.Series(np.vectorize(get_minute)(df["Datetime"].values))
df["Week"] = pd.Series(np.vectorize(get_week)(df["Datetime"].values))
df["Weekday"] = pd.Series(np.vectorize(get_weekday)(df["Datetime"].values))
df = df[df["Year"]>=2013].copy()

In [4]:
df.fillna(value={"ADDR_PCT_CD": -1.0, "LAW_CAT_CD": "UNKNOWN", "BORO_NM": "UNKNOWN",
                 "OFNS_DESC": "UNKNOWN", "PD_DESC": "UNKNOWN", "VIC_AGE_GROUP": "UNKNOWN",
                 "VIC_SEX": "UNKNOWN", "VIC_RACE": "UNKNOWN"}, inplace=True)
age_groups_all = set(np.unique(df["VIC_AGE_GROUP"].values))
age_groups = ["<18", "18-24", "25-44", "45-64", "65+", "UNKNOWN", "OTHER"]
df.loc[df["VIC_AGE_GROUP"].isin(set(age_groups_all.difference(set(age_groups)))), "VIC_AGE_GROUP"] = "OTHER"

## Define focuscrimes to use in later data creation

In [5]:
crime_counts = df[["OFNS_DESC", "Time"]].groupby(by="OFNS_DESC", as_index=False).count().sort_values(by="Time", ascending=False)
focuscrimes = set(crime_counts[(crime_counts["Time"]>5000)]["OFNS_DESC"].values)

## Extract and process the data for the desired .csv and geojson files

In [6]:
# Number of crimes in each precinct
df_precinct = df[["ADDR_PCT_CD", "OFNS_DESC"]].copy()

precinct_DF = pd.DataFrame(data={"ADDR_PCT_CD": df["ADDR_PCT_CD"].values}).drop_duplicates().dropna()
for crime in focuscrimes:
    df_temp = df_precinct[df_precinct["OFNS_DESC"]==crime]
    df_group = df_temp.groupby(by="ADDR_PCT_CD", as_index=False).count()
    df_group = df_group.rename(index=str, columns={"OFNS_DESC": crime})
    precinct_DF = precinct_DF.merge(df_group, on="ADDR_PCT_CD", how="left")

precinct_DF["Total"] = precinct_DF.sum(axis=1)
precinct_DF.to_csv(os.path.join("web_data", "crimes_by_precinct.csv"))

In [7]:
# Number of crimes for each victim age group
df_vicage = df[["VIC_AGE_GROUP", "OFNS_DESC"]].copy()
vicage_DF = pd.DataFrame(data={"VIC_AGE_GROUP": df["VIC_AGE_GROUP"].values}).drop_duplicates().dropna()

for crime in focuscrimes:
    df_temp = df_vicage[df_vicage["OFNS_DESC"]==crime]
    df_group = df_temp.groupby(by="VIC_AGE_GROUP", as_index=False).count()
    df_group = df_group.rename(index=str, columns={"OFNS_DESC": crime})
    vicage_DF = vicage_DF.merge(df_group, on="VIC_AGE_GROUP", how="left")

vicage_DF["Total"] = vicage_DF.sum(axis=1)
vicage_DF.to_csv(os.path.join("web_data", "crimes_by_vicage.csv"))

In [8]:
# Number of crimes for each victim sex
df_vicsex = df[["VIC_SEX", "OFNS_DESC"]].copy()
vicsex_DF = pd.DataFrame(data={"VIC_SEX": df["VIC_SEX"].values}).drop_duplicates().dropna()

for crime in focuscrimes:
    df_temp = df_vicsex[df_vicage["OFNS_DESC"]==crime]
    df_group = df_temp.groupby(by="VIC_SEX", as_index=False).count()
    df_group = df_group.rename(index=str, columns={"OFNS_DESC": crime})
    vicsex_DF = vicsex_DF.merge(df_group, on="VIC_SEX", how="left")

vicsex_DF["Total"] = vicsex_DF.sum(axis=1)
vicsex_DF.to_csv(os.path.join("web_data", "crimes_by_vicsex.csv"))

In [9]:
# Number of crimes per year for each precinct
df_precinct = df[(df["OFNS_DESC"].isin(focuscrimes))][["ADDR_PCT_CD", "OFNS_DESC", "Year", "Time"]].copy()
df_group = df.groupby(by=["ADDR_PCT_CD", "Year", "OFNS_DESC"], as_index=False).count()
precinct_DF_year = df_group[["ADDR_PCT_CD", "Year"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(precinct_DF_year)):
        row = precinct_DF_year.iloc[i]
        val = df_temp[(df_temp["ADDR_PCT_CD"]==row["ADDR_PCT_CD"]) & (df_temp["Year"]==row.Year)].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
        
    precinct_DF_year[crime] = pd.Series(temp)
        
precinct_DF_year["Total"] = precinct_DF_year.sum(axis=1)
precinct_DF_year.to_csv(os.path.join("web_data", "crimes_per_year_by_precinct.csv"))

In [10]:
# Number of crimes per year for each victim age group
df_vicage = df[(df["OFNS_DESC"].isin(focuscrimes))][["VIC_AGE_GROUP", "OFNS_DESC", "Year", "Time"]].copy()
df_group = df.groupby(by=["VIC_AGE_GROUP", "Year", "OFNS_DESC"], as_index=False).count()
vicage_DF_year = df_group[["VIC_AGE_GROUP", "Year"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(vicage_DF_year)):
        row = vicage_DF_year.iloc[i]
        val = df_temp[(df_temp["VIC_AGE_GROUP"]==row["VIC_AGE_GROUP"]) & (df_temp["Year"]==row.Year)].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
        
    vicage_DF_year[crime] = pd.Series(temp)
        
vicage_DF_year["Total"] = vicage_DF_year.sum(axis=1)
vicage_DF_year.to_csv(os.path.join("web_data", "crimes_per_year_by_vicage.csv"))

In [11]:
# Number of crimes per year for each victim sex
df_vicsex = df[(df["OFNS_DESC"].isin(focuscrimes))][["VIC_SEX", "OFNS_DESC", "Year", "Time"]].copy()
df_group = df.groupby(by=["VIC_SEX", "Year", "OFNS_DESC"], as_index=False).count()
vicsex_DF_year = df_group[["VIC_SEX", "Year"]].copy().drop_duplicates().dropna().reset_index(drop=True)

for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(vicsex_DF_year)):
        row = vicsex_DF_year.iloc[i]
        val = df_temp[(df_temp["VIC_SEX"]==row["VIC_SEX"]) & (df_temp["Year"]==row.Year)].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
        
    vicsex_DF_year[crime] = pd.Series(temp)
        

vicsex_DF_year["Total"] = vicsex_DF_year.sum(axis=1)
vicsex_DF_year.to_csv(os.path.join("web_data", "crimes_per_year_by_vicsex.csv"))

In [13]:
# Number of crimes per week for crime type
df_years_weeks = df[(df["OFNS_DESC"].isin(focuscrimes))][["Year", "Week", "OFNS_DESC", "Time"]].copy()
df_group = df.groupby(by=["Year", "Week", "OFNS_DESC"], as_index=False).count()

years_weeks_DF = df_group[["Year", "Week"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(years_weeks_DF)):
        row = years_weeks_DF.iloc[i]
        val = df_temp[(df_temp["Year"]==row["Year"]) & (df_temp["Week"]==row.Week)].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    years_weeks_DF[crime] = pd.Series(temp)
    
years_weeks_DF["Total"] = years_weeks_DF.sum(axis=1)
years_weeks_DF.to_csv(os.path.join("web_data", "crimes_per_week.csv"))

In [14]:
# Number of crimes per week for each precinct
df_years_weeks = df[(df["OFNS_DESC"].isin(focuscrimes))][["Year", "Week", "ADDR_PCT_CD", "Time"]].copy()
df_group = df.groupby(by=["Year", "Week", "ADDR_PCT_CD"], as_index=False).count()

years_weeks_DF = df_group[["Year", "Week"]].copy().drop_duplicates().dropna().reset_index(drop=True)
precincts = np.sort(np.unique(df["ADDR_PCT_CD"]))
for p in precincts:
    temp = []
    df_temp = df_group[df_group["ADDR_PCT_CD"]==p]
    for i in range(len(years_weeks_DF)):
        row = years_weeks_DF.iloc[i]
        val = df_temp[(df_temp["Year"]==row["Year"]) & (df_temp["Week"]==row.Week)].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    years_weeks_DF[p] = pd.Series(temp)
    
years_weeks_DF["Total"] = years_weeks_DF.sum(axis=1)
years_weeks_DF.to_csv(os.path.join("web_data", "crimes_per_week_precinct.csv"))

In [15]:
# Number of crimes per weekday for each precinct
df_years_weeks = df[(df["OFNS_DESC"].isin(focuscrimes))][["Year", "Week", "Weekday", "ADDR_PCT_CD", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["Year", "Week", "Weekday", "ADDR_PCT_CD"], as_index=False).count()

years_weeks_DF = df_group[["Year", "Week", "Weekday"]].copy().drop_duplicates().dropna().reset_index(drop=True)
precincts = np.sort(np.unique(df["ADDR_PCT_CD"]))
for p in precincts:
    temp = []
    df_temp = df_group[df_group["ADDR_PCT_CD"]==p]
    for i in range(len(years_weeks_DF)):
        row = years_weeks_DF.iloc[i]
        val = df_temp[(df_temp["Year"]==row["Year"]) & (df_temp["Week"]==row.Week) & (df_temp["Weekday"]==row.Weekday)].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    years_weeks_DF[p] = pd.Series(temp)
    
years_weeks_DF["Total"] = years_weeks_DF.sum(axis=1)
years_weeks_DF.to_csv(os.path.join("web_data", "crimes_per_weekday_precinct.csv"))

In [16]:
# Number of crimes per weekday for each crime type
df_years_weeks = df[(df["OFNS_DESC"].isin(focuscrimes))][["Year", "Week", "Weekday", "OFNS_DESC", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["Year", "Week", "Weekday", "OFNS_DESC"], as_index=False).count()

years_weeks_DF = df_group[["Year", "Week", "Weekday"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(years_weeks_DF)):
        row = years_weeks_DF.iloc[i]
        val = df_temp[(df_temp["Year"]==row["Year"]) & (df_temp["Week"]==row.Week) & (df_temp["Weekday"]==row.Weekday)].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    years_weeks_DF[crime] = pd.Series(temp)
    
years_weeks_DF["Total"] = years_weeks_DF.sum(axis=1)
years_weeks_DF.to_csv(os.path.join("web_data", "crimes_per_weekday_crime_type.csv"))

In [18]:
# Average number of crimes per day for each crime type
df_day = df[(df["OFNS_DESC"].isin(focuscrimes))][["Day", "OFNS_DESC", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["Year", "Month", "Day", "OFNS_DESC"], as_index=False).count()
df_group = df_group.groupby(by=["Day", "OFNS_DESC"], as_index=False).mean()

day_DF = df_group[["Day"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(day_DF)):
        row = day_DF.iloc[i]
        val = df_temp[(df_temp["Day"]==row["Day"])].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    day_DF[crime] = pd.Series(temp)
    
day_DF["Total"] = day_DF.sum(axis=1)
day_DF.to_csv(os.path.join("web_data", "crimes_per_dayofmonth_avg_crime_type.csv"))

In [19]:
# Average number of crimes per Month for each crime type
df_month = df[(df["OFNS_DESC"].isin(focuscrimes))][["Month", "OFNS_DESC", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["Year", "Month", "OFNS_DESC"], as_index=False).count()
df_group = df_group.groupby(by=["Month", "OFNS_DESC"], as_index=False).mean()

month_DF = df_group[["Month"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(month_DF)):
        row = month_DF.iloc[i]
        val = df_temp[(df_temp["Month"]==row["Month"])].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    month_DF[crime] = pd.Series(temp)
    
month_DF["Total"] = month_DF.sum(axis=1)
month_DF.to_csv(os.path.join("web_data", "crimes_per_month_avg_crime_type.csv"))

In [20]:
# Average number of crimes per year for each crime type
df_year = df[(df["OFNS_DESC"].isin(focuscrimes))][["Year", "OFNS_DESC", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["Year", "OFNS_DESC"], as_index=False).count()
df_group = df_group.groupby(by=["Year", "OFNS_DESC"], as_index=False).mean()

year_DF = df_group[["Year"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(year_DF)):
        row = year_DF.iloc[i]
        val = df_temp[(df_temp["Year"]==row["Year"])].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    year_DF[crime] = pd.Series(temp)
    
year_DF["Total"] = year_DF.sum(axis=1)
year_DF.to_csv(os.path.join("web_data", "crimes_per_year_avg_crime_type.csv"))

In [21]:
# Average number of crimes per week for each crime type
df_week = df[(df["OFNS_DESC"].isin(focuscrimes))][["Week", "OFNS_DESC", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["Year", "Month", "Week", "OFNS_DESC"], as_index=False).count()
df_group = df_group.groupby(by=["Week", "OFNS_DESC"], as_index=False).mean()

week_DF = df_group[["Week"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(week_DF)):
        row = week_DF.iloc[i]
        val = df_temp[(df_temp["Week"]==row["Week"])].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    week_DF[crime] = pd.Series(temp)
    
week_DF["Total"] = week_DF.sum(axis=1)
week_DF.to_csv(os.path.join("web_data", "crimes_per_week_avg_crime_type.csv"))

In [22]:
# Average number of crimes per weekday for each crime type
df_weekday = df[(df["OFNS_DESC"].isin(focuscrimes))][["Weekday", "OFNS_DESC", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["Year", "Month", "Week", "Weekday", "OFNS_DESC"], as_index=False).count()
df_group = df_group.groupby(by=["Weekday", "OFNS_DESC"], as_index=False).mean()

weekday_DF = df_group[["Weekday"]].copy().drop_duplicates().dropna().reset_index(drop=True)
for crime in focuscrimes:
    temp = []
    df_temp = df_group[df_group["OFNS_DESC"]==crime]
    for i in range(len(weekday_DF)):
        row = weekday_DF.iloc[i]
        val = df_temp[(df_temp["Weekday"]==row["Weekday"])].Time.values
        if len(val) == 0:
            temp.append(0)
        else:
            temp.append(val[0])
    
    weekday_DF[crime] = pd.Series(temp)
    
weekday_DF["Total"] = weekday_DF.sum(axis=1)
weekday_DF.to_csv(os.path.join("web_data", "crimes_per_weekday_avg_crime_type.csv"))

In [23]:
# Number of crimes per crime type
df_all = df[(df["OFNS_DESC"].isin(focuscrimes))][["OFNS_DESC", "Time"]].copy()
df_group = df[(df["OFNS_DESC"].isin(focuscrimes))].groupby(by=["OFNS_DESC"], as_index=False).count()

all_dict = {}
for crime in focuscrimes:
    all_dict[crime] = df_group[df_group["OFNS_DESC"]==crime]["Time"].values
    
all_DF = pd.DataFrame(data=all_dict)
all_DF["Total"] = all_DF.sum(axis=1)
all_DF.to_csv(os.path.join("web_data", "crimes_all_crime_type.csv"))

In [24]:
# Geographical locations of each crime along with what year and week they were committed and in what precinct

# Clustering of crime locations on a weekly basis
df_years_weeks = df[df["OFNS_DESC"].isin(focuscrimes)][["Year", "Week", "ADDR_PCT_CD", "Latitude", "Longitude", "OFNS_DESC"]].copy()
cluster_labels = []
df_1 = df_years_weeks.sort_values(by=["Year", "Week"])
df_1 = df_1[(pd.notna(df_1["Longitude"])) & (pd.notna(df_1["Latitude"]))]
for y in [2013, 2014, 2015, 2016, 2017]:
    for w in range(1,54):
        temp = df_1[(df_1["Year"]==y) & (df_1["Week"]==w)]
        if (len(temp)>0):
            X = temp[["Longitude", "Latitude"]].values
            brc = Birch(branching_factor=50, n_clusters=None, threshold=0.05, compute_labels=True)
            brc.fit(X)
            l = brc.predict(X)
            cluster_labels.extend(l)

df_1["Clusters"] = cluster_labels

# Split file into two parts because of the file size
df_1[df_1["Year"]<=2015].to_csv(os.path.join("web_data", "crime_locations_clusters_1.csv"), index=False)
df_1[df_1["Year"]>2015].to_csv(os.path.join("web_data", "crime_locations_clusters_2.csv"), index=False)